In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from datetime import datetime
import calendar
import json
from datetime import date, timedelta
from oauth2client.service_account import ServiceAccountCredentials
import setup_environment_2
import importlib
import import_ipynb
import warnings
from datetime import datetime, timedelta
import pytz  
import os
import snowflake.connector
import boto3
warnings.filterwarnings("ignore")
importlib.reload(setup_environment_2)
setup_environment_2.initialize_env()
import base64
from botocore.exceptions import ClientError
from requests import get
from pathlib import Path
import requests
import time
import gspread

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/snowflake/connector/options.py:104: UserWarning: You have an incompatible version of 'pyarrow' installed (22.0.0), please install a version that adheres to: 'pyarrow<19.0.0; extra == "pandas"'
  warn_incompatible_dep(


/home/ec2-user/.Renviron
/home/ec2-user/service_account_key.json


In [2]:
import os
import boto3
import base64
from botocore.exceptions import ClientError
import json
from requests import get
from pathlib import Path
import requests
    
def get_secret(secret_name):
    region_name = "us-east-1"

    # Create a Secrets Manager client
    session = boto3.session.Session()
    client = session.client(
        service_name='secretsmanager',
        region_name=region_name
    )

    # In this sample we only handle the specific exceptions for the 'GetSecretValue' API.
    # See https://docs.aws.amazon.com/secretsmanager/latest/apireference/API_GetSecretValue.html
    # We rethrow the exception by default.

    try:
        get_secret_value_response = client.get_secret_value(SecretId=secret_name)
    except ClientError as e:
        if e.response['Error']['Code'] == 'DecryptionFailureException':
            # Secrets Manager can't decrypt the protected secret text using the provided KMS key.
            # Deal with the exception here, and/or rethrow at your discretion.
            raise e
        elif e.response['Error']['Code'] == 'InternalServiceErrorException':
            # An error occurred on the server side.
            # Deal with the exception here, and/or rethrow at your discretion.
            raise e
        elif e.response['Error']['Code'] == 'InvalidParameterException':
            # You provided an invalid value for a parameter.
            # Deal with the exception here, and/or rethrow at your discretion.
            raise e
        elif e.response['Error']['Code'] == 'InvalidRequestException':
            # You provided a parameter value that is not valid for the current state of the resource.
            # Deal with the exception here, and/or rethrow at your discretion.
            raise e
        elif e.response['Error']['Code'] == 'ResourceNotFoundException':
            # We can't find the resource that you asked for.
            # Deal with the exception here, and/or rethrow at your discretion.
            raise e
    else:
        # Decrypts secret using the associated KMS CMK.
        # Depending on whether the secret is a string or binary, one of these fields will be populated.
        if 'SecretString' in get_secret_value_response:
            return get_secret_value_response['SecretString']
        else:
            return base64.b64decode(get_secret_value_response['SecretBinary'])

        
pricing_api_secret = json.loads(get_secret("prod/pricing/api/"))
username = pricing_api_secret["egypt_username"]
password = pricing_api_secret["egypt_password"]
secret = pricing_api_secret["egypt_secret"]

# get access token
def get_access_token(url, client_id, client_secret):
    """
    get_access_token function takes three parameters and returns a session token
    to connect to MaxAB APIs

    :param url: production MaxAB token URL
    :param client_id: client ID
    :param client_secret: client sercret
    :return: session token
    """
    response = requests.post(
        url,
        data={"grant_type": "password",
              "username": username,
              "password": password},
        auth=(client_id, client_secret),
    )
    return response.json()["access_token"]


def post_prices(id_,file_name):
    token = get_access_token('https://sso.maxab.info/auth/realms/maxab/protocol/openid-connect/token',
                             'main-system-externals',
                             secret)
    url = "https://api.maxab.info/main-system/api/admin-portal/cohorts/{}/pricing".format(id_)
    payload={}
    files=[
      ('sheet',(file_name,open(file_name,'rb'),'application/vnd.openxmlformats-officedocument.spreadsheetml.sheet'))
    ]
    headers = {
      'Authorization': 'bearer {}'.format(token)}

    response = requests.request("POST", url, headers=headers, data=payload, files=files)
    return response

def post_cart_rules(id_,file_name):
    token = get_access_token('https://sso.maxab.info/auth/realms/maxab/protocol/openid-connect/token',
                             'main-system-externals',
                             secret)
    url = "https://api.maxab.info/main-system/api/admin-portal/cohorts/{}/cart-rules".format(id_)
    payload={}
    files=[
      ('sheet',(file_name,open(file_name,'rb'),'application/vnd.openxmlformats-officedocument.spreadsheetml.sheet'))
    ]
    headers = {
      'Authorization': 'bearer {}'.format(token)}

    response = requests.request("POST", url, headers=headers, data=payload, files=files)
    return response

In [3]:
def query_snowflake(query, columns=[]):
    import os
    import snowflake.connector
    import numpy as np
    import pandas as pd
    con = snowflake.connector.connect(
        user =  os.environ["SNOWFLAKE_USERNAME"],
        account= os.environ["SNOWFLAKE_ACCOUNT"],
        password= os.environ["SNOWFLAKE_PASSWORD"],
        database =os.environ["SNOWFLAKE_DATABASE"]
    )
    try:
        cur = con.cursor()
        cur.execute("USE WAREHOUSE COMPUTE_WH")
        cur.execute(query)
        if len(columns) == 0:
            out = pd.DataFrame(np.array(cur.fetchall()))
        else:
            out = pd.DataFrame(np.array(cur.fetchall()),columns=columns)
        return out
    except Exception as e:
        print("Error: ", e)
    finally:
        cur.close()
        con.close()

In [4]:
query = '''
select 
		f.product_id,
		CONCAT(products.name_ar,' ',products.size,' ',product_units.name_ar) as sku,
		brands.name_ar as brand, 
		categories.name_ar as cat,	
		f.wac1,
		f.wac_p
from finance.all_cogs f
JOIN products on products.id=f.product_id
JOIN brands on products.brand_id = brands.id 
JOIN categories ON products.category_id = categories.id
JOIN product_units ON product_units.id = products.unit_id 
where current_timestamp between f.from_date and f.to_date 
and not categories.name_ar in (
    SELECT categories.name_ar AS cat
                    FROM categories
                    JOIN sections s ON s.id = categories.section_id
                    WHERE   categories.name_ar LIKE '%سايب%'
                        OR  categories.name_ar LIKE '%بالتة%'
                        OR  categories.section_id IN (225, 318, 285, 121, 87, 351, 417)

)
'''

wacs = query_snowflake(query, columns = ['product_id','sku','brand','cat','wac1','wac_p']) 
wacs.product_id=pd.to_numeric(wacs.product_id)
wacs.wac1=pd.to_numeric(wacs.wac1)
wacs.wac_p=pd.to_numeric(wacs.wac_p)

In [11]:
query ='''

select region,cohort_id,product_id,packing_unit_id,basic_unit_count,price
from (
select   case when cpc.cohort_id in (700,695) then 'Cairo'
             when cpc.cohort_id in (701) then 'Giza'
             when cpc.cohort_id in (704,698) then 'Delta East'
             when cpc.cohort_id in (703,697) then 'Delta West'
             when cpc.cohort_id in (696,1123,1124,1125,1126) then 'Upper Egypt'
             when cpc.cohort_id in (702,699) then 'Alexandria'
        end as region,
		cohort_id,
        pu.product_id,
		pu.packing_unit_id as packing_unit_id,
		pu.basic_unit_count,
        cpc.price as price,
		cpc.created_at,
		max(cpc.created_at)over(partition by pu.product_id ,pu.packing_unit_id,cohort_id) as max_date 
from cohort_pricing_changes cpc 
join    PACKING_UNIT_PRODUCTS pu on pu.id = cpc.product_packing_unit_id
WHERE   cpc.cohort_id in (700,701,702,703,704,696,695,698,697,699,1123,1124,1125,1126)
)x 
where created_at = max_date
'''
prices = setup_environment_2.dwh_pg_query(query, columns = ['region','cohort_id','product_id','packing_unit_id','basic_unit_count','price'])
prices.product_id=pd.to_numeric(prices.product_id)
prices.cohort_id=pd.to_numeric(prices.cohort_id)
prices.packing_unit_id=pd.to_numeric(prices.packing_unit_id)
prices.basic_unit_count=pd.to_numeric(prices.basic_unit_count)
prices.price=pd.to_numeric(prices.price)

In [6]:
query = '''
select cppu.cohort_id,product_id,packing_unit_id,basic_unit_count,COALESCE(cppu.MAX_PER_SALES_ORDER,cppu2.MAX_PER_SALES_ORDER) as current_cart_rule
from COHORT_PRODUCT_PACKING_UNITS cppu 
join PACKING_UNIT_PRODUCTS pup on cppu.PRODUCT_PACKING_UNIT_ID = pup.id 
join cohorts c on c.id = cppu.cohort_id
join COHORT_PRODUCT_PACKING_UNITS cppu2 on cppu.PRODUCT_PACKING_UNIT_ID = cppu2.PRODUCT_PACKING_UNIT_ID and cppu2.cohort_id = c.FALLBACK_COHORT_ID 
where cppu.cohort_id in (700,701,702,703,704,1123,1124,1125,1126)

'''
live_cart_rules = query_snowflake(query, columns = ['cohort_id','product_id','packing_unit_id','basic_unit_count','current_cart_rule']) 
live_cart_rules.cohort_id=pd.to_numeric(live_cart_rules.cohort_id)
live_cart_rules.product_id=pd.to_numeric(live_cart_rules.product_id)
live_cart_rules.packing_unit_id=pd.to_numeric(live_cart_rules.packing_unit_id)
live_cart_rules.basic_unit_count=pd.to_numeric(live_cart_rules.basic_unit_count)
live_cart_rules.current_cart_rule=pd.to_numeric(live_cart_rules.current_cart_rule)

In [8]:
product = 159 
recover = 1 

In [12]:
chosen_cart = live_cart_rules[(live_cart_rules['product_id']==product)]
chosen_prices = prices[(prices['product_id']==product)]

In [13]:
chosen_prices

,region,cohort_id,product_id,packing_unit_id,basic_unit_count,price
2184,Cairo,695,159,1,18,747.25
2185,Upper Egypt,696,159,1,18,747.25
2186,Delta West,697,159,1,18,747.25
2187,Delta East,698,159,1,18,736.75
2188,Alexandria,699,159,1,18,718.50
2189,Cairo,700,159,1,18,747.25
2190,Giza,701,159,1,18,747.25
2191,Alexandria,702,159,1,18,718.50
2192,Delta West,703,159,1,18,747.25
2193,Delta East,704,159,1,18,736.75


In [ ]:
to_upload = chosen_prices.copy()
to_upload=to_upload.merge(wacs ,on ='product_id')
to_upload['final_new_price'] =  (to_upload['wac_p']*prices['basic_unit_count'])/(1-0.05)
to_upload['ind'] =1 
to_upload['remove_min'] = np.nan
to_upload = to_upload[['cohort_id','product_id','sku','packing_unit_id','final_new_price','region','ind','remove_min','wac_p']]
to_upload['margin'] = (to_upload['final_new_price']-to_upload['wac_p'])/to_upload['final_new_price']
to_upload

In [ ]:
cart_rules_data = chosen_cart.copy()
cart_rules_data['tier_2']=1
cart_rules_data=cart_rules_data[['cohort_id','product_id','packing_unit_id','tier_2']]

In [ ]:
if recover:
    to_upload = chosen_prices.copy()
    to_upload=to_upload.merge(wacs ,on ='product_id')
    cart_rules_data = chosen_cart.copy()
    cart_rules_data['tier_2']=cart_rules_data['current_cart_rule']
    cart_rules_data=cart_rules_data[['cohort_id','product_id','packing_unit_id','tier_2']]
    display(to_upload,cart_rules_data)

In [ ]:
for cohort in cart_rules_data.cohort_id.unique():
    req_data = cart_rules_data[cart_rules_data['cohort_id']==cohort]
    if len(req_data) > 0 :
        req_data = req_data[['product_id','packing_unit_id','tier_2']]
        req_data.columns = ['Product ID','Packing Unit ID','Cart Rules']
        req_data.to_excel(f'CartRules_{cohort}.xlsx', index=False, engine='xlsxwriter')
        time.sleep(5)
        x =  post_cart_rules(cohort,f'CartRules_{cohort}.xlsx')
        if x.ok:
            print(f"success_{cohort}")
        else:
            print(f"ERROR_{cohort}")
            print(x.content)
            break

In [ ]:
for cohort in to_upload.cohort_id.unique():
        upload = to_upload[to_upload['cohort_id']==cohort]
        out=upload[['product_id', 'sku', 'packing_unit_id', 'final_new_price', 'ind', 'remove_min']].copy()
        out.columns = ['Product ID','Product Name','Packing Unit ID','Price','ind','remove_min']
        out['Visibility (YES/NO)'] = 'YES'
        out.loc[(out['ind'] == 1) & (out['remove_min'] == 1), 'Visibility (YES/NO)'] = 'NO'
        out.drop(columns=['ind','remove_min'], inplace=True)
        out = out.drop_duplicates()
        out['Execute At (format:dd/mm/yyyy HH:mm)'] = None
        out['Tags'] = None
        file_name_ = 'Uploads/1_new_{}.xlsx'.format(cohort).replace(' ','_')
        out.to_excel(file_name_,index = False,engine = 'xlsxwriter')
        time.sleep(5)
        x = post_prices(cohort, 'Uploads/1_new_{}.xlsx'.format(cohort).replace(' ','_'))
        if ('"success":true' in str(x.content).lower()):
            print(f"Prices are upoladed successfuly cohort:{cohort}")
        else:
            print(f"ERROR cohort: {cohort}")
            print(x.content)
            break
